<a href="https://colab.research.google.com/github/Rosefinch-Midsummer/Awesome-Colab/blob/master/sklearn/sklearn_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[docs](https://scikit-learn.org/stable/)

[apache-cn-docs](https://sklearn.apachecn.org/)



In [0]:
from sklearn.externals import joblib
#save
joblib.dump(clf, 'save/clf.pkl')

#load
clf3=joblib.load('save/clf.pkl')
print(clf3.predict(X[0:1]))

# pipeline

[Auto Machine Learning笔记 - Pipelines 制作教程](http://codewithzhangyi.com/2018/08/07/Machine%20Learning%E7%AC%94%E8%AE%B0%20-%20Pipelines%20%E5%88%B6%E4%BD%9C%E6%95%99%E7%A8%8B/)

征选择、标准化和分类。Pipeline 在这里有多种用途:

- 便捷性和封装性 你只要对数据调用 fit和 predict 一次来适配所有的一系列评估器。
- 联合的参数选择 你可以一次grid search管道中所有评估器的参数。
- 安全性 训练转换器和预测器使用的是相同样本，管道有助于防止来自测试数据的统计数据泄露到交叉验证的训练模型中。

管道中的所有评估器，除了最后一个评估器，管道的所有评估器必须是转换器。 (例如，必须有 transform 方法). 最后一个评估器的类型不限（转换器、分类器等等）

In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "naesaranderzhang" # username from the json file
os.environ['KAGGLE_KEY'] = "d580334ca87c88ae66a1995d318dca80" # key from the json file
!kaggle competitions download -c spooky-author-identification # api copied from kaggle

  0% 0.00/538k [00:00<?, ?B/s]
100% 538k/538k [00:00<00:00, 81.0MB/s]
  0% 0.00/29.4k [00:00<?, ?B/s]
100% 29.4k/29.4k [00:00<00:00, 27.5MB/s]
  0% 0.00/1.26M [00:00<?, ?B/s]
100% 1.26M/1.26M [00:00<00:00, 81.6MB/s]


In [0]:
!unzip train.zip
!unzip test.zip


Archive:  train.zip
  inflating: train.csv               
Archive:  test.zip
  inflating: test.csv                


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df_train = pd.read_csv('train.csv')

df_train.dropna(axis=0)
df_train.set_index('id', inplace = True)

df_train.head()

,text,author
id,,
id26305,"This process, however, afforded me no means of...",EAP
id17569,It never once occurred to me that the fumbling...,HPL
id11008,"In his left hand was a gold snuff box, from wh...",EAP
id27763,How lovely is spring As we looked from Windsor...,MWS
id12958,"Finding nothing else, not even gold, the Super...",HPL


文本特征预处理

以下为适用于所有文本的数据清洗操作：

- 将文本信息去标点符号，且全部用小写字母
- 计算文本长度
- 计算文本字数
- 计算 非停用词 字数
- 计算 非停用词单词的 平均长度
- 计算逗号数

In [0]:
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

stopWords = set(stopwords.words('english')) # 可能需要手动下载 stopwords

#creating a function to encapsulate preprocessing, to mkae it easy to replicate on  submission data
def processing(df):
    #lowering and removing punctuation
    df['processed'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]','', x.lower()))
    
    #numerical feature engineering
    #total length of sentence
    df['length'] = df['processed'].apply(lambda x: len(x))
    #get number of words
    df['words'] = df['processed'].apply(lambda x: len(x.split(' ')))
    df['words_not_stopword'] = df['processed'].apply(lambda x: len([t for t in x.split(' ') if t not in stopWords]))
    #get the average word length
    df['avg_word_length'] = df['processed'].apply(lambda x: np.mean([len(t) for t in x.split(' ') if t not in stopWords]) if len([len(t) for t in x.split(' ') if t not in stopWords]) > 0 else 0)
    #get the average word length
    df['commas'] = df['text'].apply(lambda x: x.count(','))

    return(df)

df_train= processing(df_train)

df_train.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,text,author,processed,length,words,words_not_stopword,avg_word_length,commas
id,,,,,,,,
id26305,"This process, however, afforded me no means of...",EAP,this process however afforded me no means of a...,224,41,21,6.380952,4
id17569,It never once occurred to me that the fumbling...,HPL,it never once occurred to me that the fumbling...,70,14,6,6.166667,0
id11008,"In his left hand was a gold snuff box, from wh...",EAP,in his left hand was a gold snuff box from whi...,195,36,19,5.947368,4
id27763,How lovely is spring As we looked from Windsor...,MWS,how lovely is spring as we looked from windsor...,202,34,21,6.476190,3
id12958,"Finding nothing else, not even gold, the Super...",HPL,finding nothing else not even gold the superin...,170,27,16,7.187500,2


创建 Pipeline

拆分训练集和测试集，输入：

In [0]:
from sklearn.model_selection import train_test_split

features= [c for c in df_train.columns.values if c  not in ['id','text','author']]
numeric_features= [c for c in df_train.columns.values if c  not in ['id','text','author','processed']]
target = 'author'

X_train, X_test, y_train, y_test = train_test_split(df_train[features], df_train[target], test_size=0.33, random_state=42)
X_train.head()

,processed,length,words,words_not_stopword,avg_word_length,commas
id,,,,,,
id19417,this panorama is indeed glorious and i should ...,91,18,6,6.666667,1
id09522,there was a simple natural earnestness about h...,240,44,18,6.277778,4
id22732,who are you pray that i duc de lomelette princ...,387,74,38,5.552632,9
id10351,he had gone in the carriage to the nearest tow...,118,24,11,5.363636,0
id24580,there is no method in their proceedings beyond...,71,13,5,7.000000,1


接下来是关键步骤。

根据特征是否为数值型，创建 两个selector transformers: TextSelector，NumberSelector

selector的作用：输入一个column，根据这个selector transformer，输出得到一个新column

简单说就是，做 data transformation，收集想要的信息，比如 text length

In [0]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

先来试一下 TextSelector 好不好用。由小变大，先创建一个mini pipeline，作用是先从数据集中抓取一列数据，再做tf-idf处理并返回结果。

创建过程只需传递一个格式如（名称，对象）的元组。括号左边是动作的名称，右边就是选取的列名。所以这个mini pipeline就是两个动作，selecting（选择一列）和tfidf-ing（对这列进行tf-idf处理）。

执行pipeline的命令，可以调用 text.fit() 来适应训练集，text.transform() 来应用于训练集，或者text.fit_transform() 来执行两者。

由于它是一个文本，它将返回一个稀疏矩阵，输入：

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

text = Pipeline([
                ('selector', TextSelector(key='processed')),
                ('tfidf', TfidfVectorizer( stop_words='english'))
            ])

text.fit_transform(X_train)

<13117x21516 sparse matrix of type '<class 'numpy.float64'>'
	with 148061 stored elements in Compressed Sparse Row format>

接下来试一下 NumberSelector 对于数值型的特征处理好不好用，同样也先建立一个mini pipeline来观察效果。

这个pipeline操作就定为简单的scaler，一列列的进行数值的StandardScaler。先以 length列为例，仍然是两个步骤，先选列，即length列，再做数值StandardScaler。（StandardScaler是数据预处理的一个常见的数值缩放操作。）

In [0]:
from sklearn.preprocessing import StandardScaler

length =  Pipeline([
                ('selector', NumberSelector(key='length')),
                ('standard', StandardScaler())
            ])

length.fit_transform(X_train)

array([[-0.50769254],
       [ 0.88000324],
       [ 2.24907223],
       ...,
       [-0.46112557],
       [-0.14447015],
       [-0.39593181]])

根据输出结果可以看出，pipeline返回一个我们想要的数值缩放矩阵。然后把剩下的数值特征列都进行缩放scaler操作。当然这个数据处理操作你可以随意更改成其他可用的。

输入：

In [0]:
words =  Pipeline([
                ('selector', NumberSelector(key='words')),
                ('standard', StandardScaler())
            ])
words_not_stopword =  Pipeline([
                ('selector', NumberSelector(key='words_not_stopword')),
                ('standard', StandardScaler())
            ])
avg_word_length =  Pipeline([
                ('selector', NumberSelector(key='avg_word_length')),
                ('standard', StandardScaler())
            ])
commas =  Pipeline([
                ('selector', NumberSelector(key='commas')),
                ('standard', StandardScaler()),
            ])

创建 FeatureUnion

pipeline管道可大可小，又大又长又粗的pipeline也是由一个个mini pipelines组成的嘛。

接下来使用FeatureUnion来连接上面做好的pipelines，形成一个类似大的pipeline。

语法操作还是格式如（名称，对象）的元组。FeatureUnion本身不是pipeline，它只是一个组合，所以需要多写一行代码，将其变为一个大pipeline。然后的事情，你懂的，还是fit，transform，或者fit_transform操作。

输入：

In [0]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion([('text', text), 
                      ('length', length),
                      ('words', words),
                      ('words_not_stopword', words_not_stopword),
                      ('avg_word_length', avg_word_length),
                      ('commas', commas)])

feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

<13117x21521 sparse matrix of type '<class 'numpy.float64'>'
	with 213646 stored elements in Compressed Sparse Row format>

甚至可以在刚刚的大pipeline尾巴上再添加一个分类器，即不仅仅是数据转化，而是增加建模/预测功能。还是原来的套路，写元组，再pipeline一下。

可以得到粗糙的 63.8%的分类精度。小试牛刀，不要太在意这些细节~

In [0]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

preds = pipeline.predict(X_test)
np.mean(preds == y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6388115134633241

再看 Pipeline

现在可以得出的结论就是，pipeline不仅能做数据预处理的流水线，更是能把整个建模套路做成流水线，只需在pipeline的结尾加上一个分类器。接下来将创建一个pipeline，完成上面所有的处理，最后用随机森林分类器。

优化 Pipeline

利用 Cross Validation 寻找更优的pipeline，就要先观察pipeline的属性，再进行超参数调参。

输入：

In [0]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'classifier', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text', 'features__length', 'features__words', 'features__words_not_stopword', 'features__avg_word_length', 'features__commas', 'features__text__memory', 'features__text__steps', 'features__text__verbose', 'features__text__selector', 'features__text__tfidf', 'features__text__selector__key', 'features__text__tfidf__analyzer', 'features__text__tfidf__binary', 'features__text__tfidf__decode_error', 'features__text__tfidf__dtype', 'features__text__tfidf__encoding', 'features__text__tfidf__input', 'features__text__tfidf__lowercase', 'features__text__tfidf__max_df', 'features__text__tfidf__max_features', 'features__text__tfidf__min_df', 'features__text__tfidf__ngram_range', 'features__text__tfidf__norm', 'features__text__tfidf__preprocessor', 'features__text__tfidf__smooth_idf', 'features__text__tfidf__stop_words'

这些都是pipeline相关的属性，即超参数，这些超参数的组合变化，超参数的数值变化都会影响一个pipeline好不好用。在此只为展示操作，因此随心情挑选四个超参数进行调优。优化方式为GridSearchCV，即 网格搜索交叉验证法，适用于少量的超参数个数和少量的数值候选调优。

In [0]:
from sklearn.model_selection import GridSearchCV

hyperparameters = { 'features__text__tfidf__max_df': [0.9, 0.95],
                    'features__text__tfidf__ngram_range': [(1,1), (1,2)],
                   'classifier__max_depth': [50, 70],
                    'classifier__min_samples_leaf': [1,2]
                  }
clf = GridSearchCV(pipeline, hyperparameters, cv=5)
 
# Fit and tune model
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('selector',
                                                                                         TextSelector(key='processed')),
                                                                                        ('tfidf',
                                                                                         TfidfVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                  

观察调优结果，即超参数最终选择的数值为多少，输入：

In [0]:
clf.best_params_

{'classifier__max_depth': 70,
 'classifier__min_samples_leaf': 2,
 'features__text__tfidf__max_df': 0.9,
 'features__text__tfidf__ngram_range': (1, 1)}

隐藏菜单操作为调用 refit，可自动使用使用pipeline来fit所有的训练数据。并将其应用于测试集。

输入：

In [0]:
#refitting on entire training data using best settings
clf.refit

preds = clf.predict(X_test)
probs = clf.predict_proba(X_test)

np.mean(preds == y_test)

0.6425255338904364

还是有一点精度的提高的。

进行预测

做模型总要有结果的，最后对数据集进行predict，看看未知文本到底是哪位作者写出来的概率更大。

输入：

In [0]:
submission = pd.read_csv('test.csv')

#preprocessing
submission = processing(submission)
predictions = clf.predict_proba(submission)

preds = pd.DataFrame(data=predictions, columns = clf.best_estimator_.named_steps['classifier'].classes_)

#generating a submission file
result = pd.concat([submission[['id']], preds], axis=1)
result.set_index('id', inplace = True)
result.head()

,EAP,HPL,MWS
id,,,
id02310,0.308413,0.278221,0.413366
id24541,0.636757,0.161106,0.202137
id00134,0.328440,0.449705,0.221855
id27757,0.647944,0.167825,0.184231
id04081,0.528074,0.206907,0.265020


Pipeline 总结

sklean提供的pipeline来将多个学习器组成流水线，通常流水线的形式为：

将数据标准化的学习器—-特征提取的学习器—-执行预测的学习器/分类器

除了最后一个学习器之外，前面的所有学习器必须提供transform方法，该方法用于数据转

（例如： 归一化，正则化，以及特征提取）